In [1]:
# Importing Necessary libraries
import pandas as pd
import numpy as np
import json
import os
import time
from tqdm.auto import tqdm
import torch
import torch.nn as nn
from sklearn.preprocessing import OneHotEncoder

/home/varun/miniconda3/envs/pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Configuration of proper device for training purposes
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")
# Path to the final model
MODEL_PATH = "bandit_bnn_model_final.pth"

Using device: cuda


In [3]:
# Reading the essential data for the model
order_data = pd.read_csv("../Datasets/order_data_cleaned.csv")
customer_data = pd.read_csv("../Datasets/customer_data_cleaned.csv")
test_data = pd.read_csv("../Datasets/test_data_question.csv")

In [4]:
# Function that converts the json_string in order_data (cleaned data) and returns a list of all items inside of it
def extract_items(order_json_string):
    try:
        data = json.loads(order_json_string)
        return [item["item_name"] for item in data["orders"][0]["item_details"]]
    except (json.JSONDecodeError, IndexError, KeyError):
        return []


# This creates a column called "item_list" which contains all the item's in a list format included in that order
order_data["item_list"] = order_data["ORDERS"].apply(extract_items)
# this creates a set (to remove duplicates) to get all unique items in order_data
all_items_in_orders = set(
    [item for sublist in order_data["item_list"] for item in sublist]
)
# this creates a set (to remove duplicates) to get all unique items in test_data
all_items_in_test = (
    set(test_data["item1"].unique())
    | set(test_data["item2"].unique())
    | set(test_data["item3"].unique())
)
# this is basically a union list of all items from both sets created to get an idea of the overall number of unique items in the complete entire database
ARM_VOCABULARY = sorted(list(all_items_in_orders | all_items_in_test))
N_ARMS = len(ARM_VOCABULARY)

# CONTEXT_FEATURES is a dictionary that maps feature names to their unique values
# this basically stores all the unique different feature values for features that are included for our valuation/prediction
CONTEXT_FEATURES = {
    "CUSTOMER_TYPE": sorted(
        customer_data["CUSTOMER_TYPE"].astype(str).unique().tolist()
    ),
    "STORE_NUMBER": sorted(order_data["STORE_NUMBER"].unique().tolist()),
    "ITEMS": ARM_VOCABULARY,
}
# this creates a OneHotEncoder for the CUSTOMER_TYPE feature
customer_type_encoder = OneHotEncoder(
    categories=[CONTEXT_FEATURES["CUSTOMER_TYPE"]],
    handle_unknown="ignore",
    sparse_output=False,
)
# this creates a OneHotEncoder for the STORE_NUMBER feature
store_number_encoder = OneHotEncoder(
    categories=[CONTEXT_FEATURES["STORE_NUMBER"]],
    handle_unknown="ignore",
    sparse_output=False,
)
# the encoders are then fitted onto the respective feature sets which are converted into a 2D array of 1 column as OneHotEncoder expects input in 2D format
customer_type_encoder.fit(np.array(CONTEXT_FEATURES["CUSTOMER_TYPE"]).reshape(-1, 1))
store_number_encoder.fit(np.array(CONTEXT_FEATURES["STORE_NUMBER"]).reshape(-1, 1))

# this creates a dictionary where each item is mapped to a unique index value (Dict so that O(1) access is possible)
ARM_MAP = {item: i for i, item in enumerate(ARM_VOCABULARY)}

In [5]:
# Main function that takes customer type, store number, and items in cart as inputs and returns a combined context vector that is provided as imput to the model for its training purposes.
def get_context_vector(customer_type, store_number, items_in_cart):
    customer_vec = customer_type_encoder.transform(np.array([[customer_type]]))
    store_vec = store_number_encoder.transform(np.array([[store_number]]))
    items_vec = np.zeros((1, len(CONTEXT_FEATURES["ITEMS"])))
    # this loop sets the appropriate indices in the items_vec to 1 for each item in the cart
    for item in items_in_cart:
        if item in ARM_MAP:
            items_vec[0, ARM_MAP[item]] = 1
    # returns the concatenated version of all feature vectors to be inputted to model.
    return np.concatenate(
        [np.array([[1]]), customer_vec, store_vec, items_vec], axis=1
    ).flatten()


# a dummy run to check proper output
dummy_context = get_context_vector("Guest", order_data["STORE_NUMBER"].iloc[0], [])
N_FEATURES = len(dummy_context)
print(f"Number of features in context vector: {N_FEATURES}")
# uncomment below line to check output
# dummy_context

Number of features in context vector: 182


In [6]:
# Custom Bandit Neural Network created for this specific use case.
# uses Dropout to ensure regularization
class BanditBNN(nn.Module):
    def __init__(self, n_features, n_arms, dropout_rate=0.5):
        super(BanditBNN, self).__init__()
        self.hidden1 = nn.Linear(n_features, 160)
        self.dropout1 = nn.Dropout(p=dropout_rate)
        self.hidden2 = nn.Linear(160, 140)
        self.dropout2 = nn.Dropout(p=dropout_rate)
        self.output = nn.Linear(140, n_arms)

    def forward(self, x):
        x = torch.relu(self.hidden1(x))
        x = self.dropout1(x)
        x = torch.relu(self.hidden2(x))
        x = self.dropout2(x)
        return self.output(x)


# Loading the trained model
model = BanditBNN(N_FEATURES, N_ARMS).to(DEVICE)
# Check for path
if os.path.exists(MODEL_PATH):
    print(f"Loading trained model from {MODEL_PATH}...")
    model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
    model.eval()
    print("Model loaded successfully.")
else:
    print(
        f"Error: Model file not found at {MODEL_PATH}. Please train and save the model first."
    )
    exit()


Loading trained model from bandit_bnn_model_final.pth...
Model loaded successfully.


/tmp/ipykernel_10640/4027396774.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))


In [7]:
# Function that returns 'top_n' item recommendations based on customer context and items in cart along with the store number.
def get_recommendations_gpu(customer_type, store_number, items_in_cart, top_n=3):
    context_vector = get_context_vector(customer_type, store_number, items_in_cart)
    context_tensor = torch.FloatTensor(context_vector).to(DEVICE)

    # Enable dropout for Monte Carlo Dropout which resembles Thompson Sampling.
    model.train()
    with torch.no_grad():
        scores = model(context_tensor)

    # Get top_n recommendations
    _, top_indices = torch.topk(scores, top_n)

    # Get the item names for the top_n indices
    return [ARM_VOCABULARY[i] for i in top_indices.cpu().numpy()]


In [18]:
# TEST FOR CONTEXT_ITEMS = 2

# THIS CELL HERE TESTS THE MODELS RECALL@3 SCORE CAPABILITY ON THE TEST_DATA WITH 2 CART ITEMS AS CONTEXT (item_1 and item_2) AND STORING THE LAST ITEM (item_3) AS GROUND TRUTH TO TEST FOR RECALL@3

# Variable to count hits
hits = 0
total = len(test_data)
# a list to get the average inference time taken to process one order
inference_times = []

if total > 0:
    for _, row in tqdm(
        test_data.iterrows(), total=total, desc="Evaluating BNN Test Set"
    ):
        # storing the first two items as context
        context_cart = [row["item1"], row["item2"]]
        ground_truth_item = row["item3"]

        # Measure Inference Time
        start_time = time.perf_counter()
        recommendations = get_recommendations_gpu(
            row["CUSTOMER_TYPE"], row["STORE_NUMBER"], context_cart
        )
        end_time = time.perf_counter()
        inference_times.append(end_time - start_time)

        if ground_truth_item in recommendations:
            hits += 1

    # --- Calculate and Print Results ---
    recall_at_3 = hits / total
    avg_inference_time_ms = (sum(inference_times) / total) * 1000

    print("\n--- BNN Evaluation Complete ---")
    print(f"Recall@3 Score: {recall_at_3:.4f} ({hits}/{total} hits)")
    print(f"Average Inference Time per Order: {avg_inference_time_ms:.2f} ms")
else:
    print("test_data_question.csv is empty. No evaluation performed.")

Evaluating BNN Test Set: 100%|██████████| 1000/1000 [00:00<00:00, 1949.44it/s]


--- BNN Evaluation Complete ---
Recall@3 Score: 0.3020 (302/1000 hits)
Average Inference Time per Order: 0.47 ms


# Creating the .xlsx and .csv file for evaluation

In [20]:
all_recommendations = []
for _, row in tqdm(
    test_data.iterrows(), total=len(test_data), desc="Generating Recommendations"
):
    context_cart = [row["item1"], row["item2"], row["item3"]]

    recommendations = get_recommendations_gpu(
        row["CUSTOMER_TYPE"], row["STORE_NUMBER"], context_cart
    )
    all_recommendations.append(recommendations)

test_data["RECOMMENDATION_1"] = [rec[0] for rec in all_recommendations]
test_data["RECOMMENDATION_2"] = [rec[1] for rec in all_recommendations]
test_data["RECOMMENDATION_3"] = [rec[2] for rec in all_recommendations]

print("Recommendation generation complete.")

output_path_xlsx = os.path.join("submission.xlsx")
output_path_csv = os.path.join("submission.csv")

submission_df = test_data[
    [
        "CUSTOMER_ID",
        "STORE_NUMBER",
        "ORDER_ID",
        "ORDER_CHANNEL_NAME",
        "ORDER_SUBCHANNEL_NAME",
        "ORDER_OCCASION_NAME",
        "CUSTOMER_TYPE",
        "item1",
        "item2",
        "item3",
        "RECOMMENDATION_1",
        "RECOMMENDATION_2",
        "RECOMMENDATION_3",
    ]
].copy()

print(f"\nSaving submission file to {output_path_xlsx}...")
submission_df.to_excel(output_path_xlsx, index=False)

print(f"Saving submission file to {output_path_csv}...")
submission_df.to_csv(output_path_csv, index=False)

print("\nProcess finished successfully.")
print(submission_df.head())

Generating Recommendations: 100%|██████████| 1000/1000 [00:00<00:00, 1903.71it/s]


Recommendation generation complete.

Saving submission file to submission.xlsx...
Saving submission file to submission.csv...

Process finished successfully.
   CUSTOMER_ID  STORE_NUMBER    ORDER_ID ORDER_CHANNEL_NAME  \
0    997177535          4915  9351345556            Digital   
1    345593831           949  3595377080            Digital   
2    160955031          2249  4071757785            Digital   
3    890671991          4154  3931766769            Digital   
4     73989021          4094  3739700809            Digital   

  ORDER_SUBCHANNEL_NAME ORDER_OCCASION_NAME CUSTOMER_TYPE  \
0                   WWT                ToGo         Guest   
1                   WWT                ToGo    Registered   
2                   WWT                ToGo         Guest   
3                   WWT                ToGo         Guest   
4                   WWT                ToGo    Registered   

                      item1                item2                     item3  \
0         Chicken 